In [2]:
%matplotlib inline
from matplotlib import pyplot as plt
import torch
import numpy as np
import imageio
import os
import csv
import torch.optim as optim
import torch.nn as nn
from collections import OrderedDict
from torchvision import datasets
from torchvision import transforms
import torch.nn.functional as F
import datetime

# 9장 - 암과 싸워 이기기 위한 파이토치 활용

In [1]:
# 2부의 전반적인 계획에서 출발해 이후 이어지는 장에서 다룰 내용의 규모와 범위에 대해 파악
# 이어지는 장에서 필요한 내용을 준비하기 위해 동작할 주변 환경 파악
# 데이터 포맷, 데이터 소스, 문제 영역의 제약 사항 파악

## 1절 - 사용 사례 소개

In [3]:
# 현실에서는 예상한 대로 일이 흘러가지 않는 경우 다룰 수 있는 도구 확인
# 수행하는 프로젝트의 성능이 생각봗 나쁠 때 해야할 일

In [4]:
# 전문적인 상황 설정: 환자의 흉부 CT 스캔 입력받아 악성 종양 자동 탐지

In [5]:
# 프로세스를 자동화하게 되면 처음부터 더 많은 작업을 수행해야 하는 비협조적인 환경 조우

In [6]:
# 폐 종양 감지 문제를 선택한 이유
# 1. 아직 미해결 과제라는 점(파이토치를 이용해 문제 해결 능력을 입증하고 파이토치에 대한 신뢰도를 높이기 위함)
# 2. 많은 팀이 도전 중이고 더욱 유망한 결과를 보인다는 점 -> 문제 해결에 너무 많은 시간 들이지 않아도 됨

In [7]:
# 2부에서는 폐 종양 탐지 문제에 집중하지만 다루는 기술은 보편적임
# 어떤 프로젝트든 훈련을 위해 데이터 조사 및 전처리 후 표현 방법 학습은 중요

## 2절 - 대규모 프로젝트 준비

In [8]:
# 이번 프로젝트는 해상도를 낮추는 다운샘플링 후 반복되는 컨볼루션 층이 모델의 대부분을 채움
# 입력이 3차원이므로 파이토치 도구도 3차원용으로 바꿔야 함

In [9]:
# 앞의 장까지와는 달리 모델 외의 부분에 노력이 얼마나 들어가는지와 관련됨
# 모델에 바로 사용할 수 있는 훈련 샘플 제공하는 이미 구축된 라이브러리 형태의 표준 데이터 없음
# 다룰 데이터 배우고 직접 구현 필요

In [10]:
# 과정이 끝나도 CT 영상 데이터를 텐서로 변환해 결과 확인 불가능
# 제한된 데이터 접근성과 연산 자원, 효과적인 모델 설계하는 능력의 한계 등의 교란 변수로 인해 생각보다 복잡함

In [11]:
# 일련의 단순한 문제들 풀고 이를 종합하여 최종적으로 완성된 결과 도출할 예정
# 모든 문제를 이렇게 풀 필요는 없으나 좋은 시작점

In [12]:
# 해결 할 문제 영역에 대해 공부하는 것은 매우 중요
# 해당 분야에 대한 통찰력과 신경망의 동작에 대한 직관력을 잘 조합하여 제대로 훈련된 실험과 정제를 통해 정보를 얻음으로써 해결책에 가까워짐

## 3절 - CT 스캔이란

In [14]:
# 이번 프로젝트에서 주요 데이터 형식으로 CT 스캔 데이터 다룸
# CT 스캔이라는 데이터 포맷의 장단점과 기본적인 특성 파악해야 활용 가능
# 단일 채널의 3차원 배열로 표현되는 3차원 엑스레이로 볼 수 있음
# 흑백 PNG 이미지 쌓아놓은 것과 같음

In [16]:
# CT 스탠 내 각 복셀은 해당 위치에 있는 물체의 평균 질량 밀도를 나타내는 숫자 값
# 밀도 높으면 하얀색, 낮으면 검은색, 지방이나 조직은 밝기 다양한 회색
# 엑스레이와의 근본적인 차이점은 3차원 형태 보존함
# 관심 없는 조직을 숨겨 대상의 내부 확인 가능

In [17]:
# 데이터가 디지털 포맷임: 스캔 과정의 원본 출력은 사람이 육안으로 구분하지 못해 컴퓨터를 사용해 재해석해야함
# CT 스캐너 설정에 따라 출력 데이터 크게 달라짐

In [18]:
# CT 스캐너가 머리부터 발에 이르는 축을 따라 거리를 측정하는 방법은 신체의 다른 두 가지 축과는 다름 -> 복셀이 정육면체가 아님
# 프로젝트 시작 전 데이터의 세부사항 구체적으로 조사할 필요 존재

## 4절 - 프로젝트: 엔드투엔드 폐암 진단기

In [19]:
# CT 스캔의 기본 확인했으니 프로젝트 구성에 대해 고민
# 디스크의 대부분은 밀도 정보 가진 CT 스캔의 3차원 배열 저장하는 데 사용할 예정
# 모델은 주로 이 3차원 배열 부분에 해당하는 여러 단면 사용

In [22]:
# 폐암 진단까지 5 가지 주요 단계
# 1. 데이터 로딩
# 2. 세그멘테이션(segmentation, 관심있는 복셀 표시)
# 3. 그룹화
# 4. 분류
# 5. 결절 분석 및 진단

In [23]:
# 결절의 위치 정보는 CT 복셀 데이터 뒤에 붙어 후보가 되고 모델에 의해 실제 결절인지 최종적으로 악성이 될지 여부를 조사
# 각각의 결절별 분류 결과는 환자 진단 결과로 병합 가능

In [24]:
# 1. 데이터 로딩
# 원본 CT 스캔 데이터를 파이토치로 사용할 수 잇는 형태로 읽음

In [25]:
# 2. 세그멘테이션
# 세그멘테이션이라는 기술 구현을 위해 파이토치를 사용하여 폐의 잠재적 결절에 해당하는 복셀 찾기
# 히트맵(heatmap)을 작성하는 것과 유사한데 3단계의 분류기에서 사용
# 관심 없는 부위 무시하고 폐 내부의 잠재적 종양에 더 집중
# 학습 중에는 단일한 작은 작업에 집중하는 게 최선

In [26]:
# 3. 그룹화
# 관심 있는 복셀들을 덩어리(후보 결절)로 묶기
# 히트맵에서 각 핫스팟의 개략적인 중심부 찾는 과정 진해
# 각 결절은 인덱스가 부여되며 중심부에서 시작하는 열과 행으로 위치 파악 가능
# 이런 제약을 통해 단순해진 문제를 최종 분류기에 전달
# 복셀 그룹화가 직접적으로 파이토치를 요구하지 않아 별도 단계 진행

In [27]:
# 4. 분류
# 3차원 컨볼루션으로 각 후보 결절에 대해 실제 결절인지 아닌지 분류
# 후보 구조로부터 종양 여부를 결정하는 것은 종양 진단에 국한된 문제
# 입력 데이터를 거를 것과 너무 거르는 것 사이에서 균형 잡을 필요성 존재

In [28]:
# 5. 결절 분석 및 진단
# 결절이 양성인지 악성인지를 영상 데이터에만 의존하여 판별
# 결절별로 악성 예측을 구한 후 이들의 최댓값을 사용하는 방식으로 하나라도 악성이 예측된 경우 해당 환자는 암 소견을 보인다고 진단
# 이 프로젝트에서는 "의심스러운 점은 없는가"을 알아보는 것이므로 단순히 최댓값을 집계

In [29]:
# 3단계와 4단계에서 훈련을 위해 사용하는 데이터는 사람이 확인하고 표기한 정답 제공
# 따라서 2,3단계(각 복셀 확인하고 결절 후보군으로 묶기)를 4단계(결절 후보 분류)와 완전히 다른 별개의 프로젝트로 취급 가능
# 전문가들이 직접 데이터에서 결절의 위치를 표시했으므로 2, 3단계와 4단계를 순서에 상관없이 개별로 진행 가능

In [31]:
# 문제를 나누어 각 단계를 독립적으로 작업하면 솔류션의 모듈화를 독려할 수 있어 유용함
# 구성원 간 작업 분배도 쉬움
# 다량한 사용 사례에 적용할 때에도 단일 시스템보다 쉬움

In [32]:
# 잠재적인 최적화나 개선을 위해 데이터에 대한 직관력 활용 능력 필요
# 딥러닝 프로젝트에서 일반적으로 요구되는 능력

In [33]:
# 결절이란 폐 내부에서 나타날 수 있는 무수한 돌기나 덩어리
# 양성 종양 또는 암이라고도 부르는 악성 종양으로 밝혀질 수 잇음
# 방사선학 관점에서 감염이나 염증, 혈액 공급 이상, 기형적 혈관, 종양 외의 질환 등 다양한 원인으로 유발된 덩어리

In [34]:
# 찾아내고자 하는 암은 항상 결절의 형태를 띨 것이며 밀도가 매우 낮은 폐 조직에 매달려 잇거나 폐 벽에 붙어 있음
# 따라서 모든 조직 검사할 필요 없음
# 예상 입력 범위를 좁혀 분류기가 작업 학습하는 것을 도움

In [35]:
# 보편적으로 사용되는 기술이지만 적용하려는 분야에 대한 이해도 필요

In [36]:
# 결절마다 크기가 천차만별이며 절반 이상은 악성이 아님

In [37]:
# 우리가 볼 CT 스캔은 LUNA(LUng Nodule Analysis, 폐 결절 분석) 그랜드 챌린지에서 가져온 것
# 다량의 폐 결절이 있는 환자의 CT 스캔 자료를 고품질로 레이블 작업한 공개 데이터셋
# 데이터 분류기 성능에 대한 공개 랭킹 제공
# 프로젝트 팀은 표준화된 기준에 대해 자신들의 탐지 방법이 효과적인지 테스트 가능
# 책에서는 LUNA 2016 데이터셋 사용

In [38]:
# LUNA 데이터는 대략 120GB 데이터
# 필요한 부분만 읽도록 데이터를 작은 덩어리로 나눠둘 100GB 정도의 캐시 공간 필요(각 장마다 캐시 공간 필요. 각 장 끝날 때마다 데이터 삭제해 공간 확보하면 됨)

In [39]:
# 사용할 데이터는 10개의 서브셋으로 구성되어 각기 subset0~9 까지의 이름 가짐
# 각각의 압축 풀면 개별 서브 디렉토리 나타날 것임
# 윈도우 사용자는 7-Zip 웹사이트에서 압축 해제 프로그램 받을 수 있음

In [ ]:
# 추가로 candidates.csv와 annotation.csv 파일 필요

## 5절 - 결론

In [ ]:
# 폐암 진단 프로젝트를 큰 맥락에서 이해하고 2부 학습을 위한 프로젝트의 방향과 구조에 대한 개요 파악